# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

In [2]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.sklearn import SKLearn

ws = Workspace.get(name="quick-starts-ws-131494")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_DS3_V2"
compute_name="notebook131494"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())





Workspace name: quick-starts-ws-131494
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-131494


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
#ws = Workspace.from_config()

# choose a name for experiment
#experiment_name = 'your experiment name here'

#experiment=Experiment(ws, experiment_name)

#workspace = Workspace(subscription_id, resource_group, workspace_name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./', target_path='data')

# create a dataset referencing the cloud location
#

Uploading an estimated of 10 files
Uploading ./AutoMLCapstone.ipynb
Uploaded ./AutoMLCapstone.ipynb, 1 files out of an estimated total of 10
Uploading ./final.ipynb
Uploaded ./final.ipynb, 2 files out of an estimated total of 10
Uploading ./log_model.pkl
Uploaded ./log_model.pkl, 3 files out of an estimated total of 10
Uploading ./train.py
Uploaded ./train.py, 4 files out of an estimated total of 10
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 5 files out of an estimated total of 10
Uploading ./HyperParameterTuningCapstone.ipynb
Uploaded ./HyperParameterTuningCapstone.ipynb, 6 files out of an estimated total of 10
Uploading ./.ipynb_checkpoints/AutoMLCapstone-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/AutoMLCapstone-checkpoint.ipynb, 7 files out of an estimated total of 10
Uploading ./.ipynb_checkpoints/final-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/final-checkpoint.ipynb, 8 files out of an estimated total of 10

$AZUREML_DATAREFERENCE_f165045fd8d0455fbb0d4daf4bad92a8

In [4]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/heart_failure_clinical_records_dataset.csv'))])

In [5]:
dataset

{
  "source": [
    "('workspaceblobstore', 'data/heart_failure_clinical_records_dataset.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [6]:
x_df=dataset.to_pandas_dataframe()

In [7]:
x_df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [6]:
x_df.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [7]:
x_df=x_df.drop(['DEATH_EVENT'],axis=1)

In [ ]:
"""
est = SKLearn(source_directory="./", 
                compute_target=compute_target, 
                vm_size=vm_size,
                entry_script='train.py'
               )

"""

In [8]:
dataset

{
  "source": [
    "('workspaceblobstore', 'data/heart_failure_clinical_records_dataset.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here



from azureml.train.automl import AutoMLConfig
import numpy as np


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=compute_target,
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name='DEATH_EVENT',
    #allowed_models=["LightGBM","RandomForest"],
    n_cross_validations=3)


In [9]:
exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib
from azureml.widgets import RunDetails
RunDetails(automl_model).show()
best_run, fitted_model = automl_model.get_output()
print(best_run)
print(fitted_model)
from azureml.core.model import Model
import joblib
joblib.dump(fitted_model,"automl_best_model.pkl")


Running on remote.
No run_configuration provided, running on notebook131494 with default configuration
Running on remote compute: notebook131494
Parent Run ID: AutoML_4472f920-4aeb-4187-a1b1-38c1ee211fd5

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputa

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: automl_test,
Id: AutoML_4472f920-4aeb-4187-a1b1-38c1ee211fd5_23,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_leaf_nodes=None,
                                                                                                    max_samples=None,
                            

['automl_best_model.pkl']

In [16]:
input_data={"data":[{'age': 75.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 582,
  'diabetes': 0,
  'ejection_fraction': 20,
  'high_blood_pressure': 1,
  'platelets': 265000.0,
  'serum_creatinine': 1.9,
  'serum_sodium': 130,
  'sex': 1,
  'smoking': 0,
  'time': 4}]}

In [12]:
dictionary =best_run.get_environment()

In [21]:
dictionary.environment_variables

{'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'}

In [25]:
dictionary.save_to_directory(path="./a")

In [20]:
best_run.get_environment()

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20201113.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "AutoML-AzureML-AutoML",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "cond

In [13]:
import json
json_object = json.dumps(dictionary, indent = 4)   
print(json_object)

TypeError: Object of type 'Environment' is not JSON serializable

In [12]:
input_data=x_df[0:2].to_dict(orient='records')
import json
import joblib
i=json.dumps(input_data)
data = json.loads(i)
model2=joblib.load("automl_best_model.pkl")
result = model2.predict(pd.DataFrame(data))

In [14]:
from azureml.core.model import Model
import joblib
model = Model.register(workspace=ws, model_path="automl_best_model.pkl", model_name="test")

Registering model test


In [16]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
local_config = LocalWebservice.deploy_configuration(port=9000)
inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

local_service = Model.deploy(ws, "test", [model], inference_config, local_config)

local_service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_9cc1a26fb8a5f7bac5f3a2cd8086e190
 ---> 614b72a6c7ae
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 561595297591
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImRhNzc1Y2I5LTljYTYtNDk0My1hZDIxLTI2ZGZhOTk1MjZmYyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEyOTQxNyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEyOTQxNyIsIndvcmtzcGFjZUlkIjoiYWNlNmMxZGUtZDU1NS00NmIxLTgwNjItZDBmMWYzMGE1NDRkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 53c8e44ef072
 ---> 9370659dca44
Step 4/5 : RUN mv '/var/azureml-app/tmpoohal_t_.py' /var/azureml-app/main.py
 ---> Running in 592f0d8fbde8
 ---> 4b3027f306b0
Step 5/5 : 

In [61]:
import json
try:
    i=json.dumps(input_data)
    data = json.loads(i)
except Eception as e:
    print(e)
#data = json.loads(i)

In [42]:
i

'{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}]}'

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'custom-env-service'

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [20]:
service.get_logs()

'2020-12-06T03:49:23.9642703Z stdout F 2020-12-06T03:49:24,206367800+00:00 - gunicorn/run \n2020-12-06T03:49:23.9692836Z stdout F 2020-12-06T03:49:24,209210700+00:00 - iot-server/run \n2020-12-06T03:49:23.9722702Z stdout F 2020-12-06T03:49:24,214625300+00:00 - rsyslog/run \n2020-12-06T03:49:24.0032773Z stdout F 2020-12-06T03:49:24,245805800+00:00 - nginx/run \n2020-12-06T03:49:24.0092728Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T03:49:24.0092728Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T03:49:24.0142819Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T03:49:24.0142819Z stderr F /usr/sbi

In [21]:
service.scoring_uri

'http://3aca9197-7e92-4032-9f35-6ba286844d32.southcentralus.azurecontainer.io/score'

In [22]:
!az login
service.update(auth_enabled=True)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F4BTNW4T2 to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "id": "da775cb9-9ca6-4943-ad21-26dfa99526fc",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure Sponsorship - Udacity -46",
    "state": "Enabled",
    "tenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "user": {
      "name": "odl_user_129417@udacitylabs.onmicrosoft.com",
      "type": "user"
    }
  }
]


In [90]:


headers={"Content-Type":"application/json"}
key="nmzf01o9jlalITeEPXejCGkw7Bqs0Iv0"
headers["Authorization"]=f'Bearer "hL84fRidVsQHrZsgj9he798pG7tEyr6c"'
import requests
result=requests.post('http://3aca9197-7e92-4032-9f35-6ba286844d32.southcentralus.azurecontainer.io/score',i,headers=headers)

In [13]:
result

array([1, 1])

In [23]:

service.delete()

compute_target.delete()